# Converting ASDF Rotations to Quaternions

This notebook shows the same thing as the
[notebook about rotation matrices](rotation-matrices.ipynb),
just using quaternions instead of rotation matrices.
For more details, have a look over there.

In [ ]:
import sympy as sp

In [ ]:
from sympy.algebras import Quaternion

In [ ]:
alpha, beta, gamma = sp.symbols('alpha beta gamma')

## Azimuth: Rotation around the z-Axis

In [ ]:
q_z = Quaternion.from_axis_angle((0, 0, 1), alpha)
q_z

Example:
Rotating the y unit vector (i.e. “looking straight ahead”)
by 90 degrees to the left:

In [ ]:
Quaternion.rotate_point((0, 1, 0), q_z.subs(alpha, sp.pi / 2))

As expected, this yields the negative x unit vector, which points westwards.

## Elevation: Rotation around the (local) x-Axis

In [ ]:
q_x = Quaternion.from_axis_angle((1, 0, 0), beta)
q_x

Example:
Applying 90 degrees of elevation to
the y unit vector:

In [ ]:
Quaternion.rotate_point((0, 1, 0), q_x.subs(beta, sp.pi / 2))

As expected, this yields a vector pointing up.

## Roll: Rotation around the (local) y-Axis

In [ ]:
q_y = Quaternion.from_axis_angle((0, 1, 0), gamma)
q_y

Example: Applying a roll angle of 90 degrees to a vector pointing up:

In [ ]:
Quaternion.rotate_point((0, 0, 1), q_y.subs(gamma, sp.pi / 2))

As expected, this yields a vector pointing east.

## Combining all Axes

This is easy,
we only have to make sure to use the right order.
As with rotation matrices,
you should read this from right to left:

In [ ]:
q = q_z * q_x * q_y
q

If you want to copy-paste this:

In [ ]:
print(q)

But you should probably pre-calculate the used terms
in order to avoid repeated evaluation of the same functions.
You could try something like this, for example:

In [ ]:
q.subs([
    (sp.sin(alpha/2), sp.symbols('s_alpha')),
    (sp.sin(beta/2), sp.symbols('s_beta')),
    (sp.sin(gamma/2), sp.symbols('s_gamma')),
    (sp.cos(alpha/2), sp.symbols('c_alpha')),
    (sp.cos(beta/2), sp.symbols('c_beta')),
    (sp.cos(gamma/2), sp.symbols('c_gamma')),
])

In [ ]:
print(_)

## Quaternion to Rotation Matrix

Just to make sure the result is the same as in the
[notebook about rotation matrices](rotation-matrices.ipynb#Combining-all-Axes),
let's calculate the rotation matrix from our quaternion.

For some reason, SymPy seems to need two simplification steps for this ...

In [ ]:
R = sp.trigsimp(sp.trigsimp(q.to_rotation_matrix()))
R